# Python for High Performance Computing
# The <span style="font-family: Courier New, Courier, monospace;">mpi4py</span> library
<hr style="border: solid 4px green">
<br>
<center> <img src="../../images/arc_logo.png"; alt="Logo" style="float: center; width: 20%"></center>
<br>
## http://www.arc.ox.ac.uk
## support@arc.ox.ac.uk

## Setup
<hr style="border: solid 4px green">

### Tools
* the `mpi4py` Python module
* an underlying MPI library
<br><br>

### Install (the Anaconda distribution)
* install `mpi4py` plus a MPI library (usually the OpenMPI or MPICH implementation)
```bash
local-shell> conda install --channel mpi4py mpich mpi4py
```
* *option*: use a `conda` environment to install `mpi4py` and the associated MPI implementation
  * avoid name clashes in `PATH`
  * see http://conda.pydata.org/docs/using/envs.html
<br><br>

### MPI and the iPython notebook
* *bad*: use `mpi4py` directly in notebook cells
* *better*: use the `ipyparallel` package
* *easiest* (this presentation): use `mpi4py` in scripts and rely on shell escape to run parallel examples

## Learning MPI
<hr style="border: solid 4px green">

### This presentation assumes (at least some) familiarity with MPI
<br><br>

### Many options for learning MPI
* traditional courses
  * ARC course dedicated to MPI
  * Archer training
* plenty of online material
  * *e.g.* http://www.archer.ac.uk/training/online/
<br><br>

### Advice: familiarise yourselves with MPI using C or Fortran
* material is generally a lot more detailed than Python documentation
* a lot more online tutorials and good textbooks than for Python
* clearer learning path than for Python (MPI for Python has several interfaces)

## What is MPI?
<hr style="border: solid 4px green">

### One standard
* **M**essage **P**assing **I**nterface
* defines the syntax and semantics of a core of library routines designed for writing portable message-passing programs (in C and Fortran)
* the *de facto* standard for distributed parallel programming
<br><br>

### Many implementations
* open source, *e.g.* OpenMPI, Mvapich2, MPICH
* commercial, *e.g.* IntelMPI
<br><br>

### Many interfaces
* C, Fortran (the MPI primary target)
* Java, Python (late additions)

## Message passing paradigm
<hr style="border: solid 4px green">

### Distributed memory programming model
* independent processes running concurrently
* all processes execute the *same* program
* each process has its own memory address space
* processes usually need some data from each other
  * the data is passed explicitly
  * data (message) passing is achieved via MPI programming
<br><br>

### Distributed computing *and* distributed memory
* the problem data is partitioned, each processes owns a partition and works on it
* computation becomes *possible* because large *memory* requirements are *distributed*
* computation becomes *faster* because large *computational* requirements are *distributed*

## Message passing paradigm (cont'd)
<hr style="border: solid 4px green">

### Advantage #1: universality & portability
* works everywhere and matches all hardware
  * separate processors connected by any network (*e.g.* modern clusters, *ad hoc* networks of computers)
  * shared memory systems (*e.g.* your laptop)
* source code is universally portable (with few exceptions, *e.g.* parallel I/O)
  * between computer systems
  * between implementations
<br><br>

### Advantage #2: performance & scalability
* the most compelling reason why MPI remains a permanent component of HPC
* with future core count increase, distributed memory computing is the key to high performance

## MPI for Python
<hr style="border: solid 4px green">

### MPI and Python
* the MPI standard says nothing about Python
* any MPI solution for Python mimics the C/C++ standard
> *Note*: the C++ interface was removed from the version 3 standard.
<br><br>

### Several options
* old: `pypar`, `pyMPI`, Scientific Python
* newer and better: `mpi4py`
<br><br>

### <span style="font-family: Courier New, Courier, monospace;">mpi4py</span> combines the advantages of both
* an interface very similar to the MPI-2 standard C++ interface
* focus is in translating MPI syntax and semantics (if you know MPI, `mpi4py` is "obvious")
* can communicate memory-contiguous data (as C/Fortran) and can communicate Python objects
* performance not the same as for C/Fortran but what is lost in performance is gained in code development time

## MPI for Python (cont'd)
<hr style="border: solid 4px green">

### Observations
* some appreciation of the object-oriented nature of Python programming is useful
* the C++ bindings in the MPI-2 standard can provide a useful quick reference for `mpi4py`
* the C++ function names can be slightly different from the corresponding C/Fortran bindings
  * often, they are reversed, *e.g.* `MPI_Buffer_attach()` becomes `MPI.Attach_buffer()`
* the iPython help `help(mpi4py.MPI)` is rather long
  * useful to narrow it down, *e.g.*, `help(mpi4py.MPI.Intracomm)`
  * this requires that you know what you are looking for

## Example: "hello world!"
<hr style="border: solid 4px green">


In [ ]:
# first, run the example executing the cell
# %load helloworld.py
"""Hellow world!  Importing and using MPI.COMM_WORLD"""

import sys
from mpi4py import MPI

def report (communicator):
    """Report rank and size of this communicator"""
    rank = communicator.rank
    size = communicator.size
    sys.stdout.write ("Hello from rank {:2d} of {:2d}\n".format(rank, size))

if __name__ == "__main__":
    """Execute in MPI.COMM_WORLD"""
    report (MPI.COMM_WORLD)


In [1]:
# second, run the example using the MPI launcher
! mpirun -np 4 python helloworld.py

Hello from rank  0 of  4
Hello from rank  1 of  4
Hello from rank  2 of  4
Hello from rank  3 of  4


## <span style="font-family: Courier New, Courier, monospace;">mpi4py</span> basics
<hr style="border: solid 4px green">

### The Python script is started through the MPI launcher <span style="font-family: Courier New, Courier, monospace;">mpirun</span>
* `mpirun` is part of the MPI library implementation (nothing to do with Python)
* used to launch *any* MPI code, *e.g.* code compiled from C/C++/Fortran source
* two principal command line arguments
  * the number of processes to launch
  * (optional) the hosts on which to run these processes (default: local host)
<br><br>

### The launcher starts and coordinates a number of processes
* all these processes run concurrently
* all processes execute the *same* Python code
* programming differentiates between what processes do

## <span style="font-family: Courier New, Courier, monospace;">mpi4py</span> basics (cont'd)
<hr style="border: solid 4px green">

### MPI initialisation
* import the `mpi4py` module
```python
from mpi4py import MPI
```
* the `import mpi4py` statement is responsible for initialising MPI (if not already initialised)
  * no analogue to the C/Fortran calls to `MPI_Init()` and `MPI_Finalize()`
  * the underlying MPI library ensures that
    * `MPI_Init()` is implicit to the `mpi4py` module being imported
    * `MPI_Finalize()` is implicit when Python process terminates
<br><br>

### The class <span style="font-family: Courier New, Courier, monospace;">MPI</span>
* loaded from module `mpi4py`
* provides the pre-defined communicator `COMM_WORLD`
* more about communicators in a moment...
<br><br>

### Size and rank
* `size = MPI.COMM_WORLD.size` (or `MPI.COMM_WORLD.Get_size()`) is the number of the processes started through `mpirun`
* `rank = MPI.COMM_WORLD.rank` (or `MPI.COMM_WORLD.Get_rank()`) is the ID of the process: $0\leq \text{rank}\leq\text{size}-1$

## Communicators
<hr style="border: solid 4px green">

### A communicator provides the context within which communication takes place
* two or more processes can pass messages between each other *only* if they belong to the same communicator
* all communicators are Python objects in `mpi4py`
* the pre-defined communicator `COMM_WORLD` contains *all* processes started by `mpirun` and is accessed via the class `MPI`
<br><br>

### Class hierarchy for communicators
```
Comm
    Intracomm
    Intercomm
        Topocomm
            Cartcomm
            Distgraphcomm
            Graphcomm
```
* example: a Cartesian communicator `Cartcomm` can be derived from `COMM_WORLD`
<br><br>

### Many methods are implemented on `Comm` and inherited by subclasses
* example
```
    comm.Barrier()
```

## Example
<hr style="border: solid 4px green">

### Creating a Cartesian communicator
`Cartcomm` object created from an existing `Intracomm` object uses, *e.g.*
```
Intracomm.Create_cart (self, dims, periods=None, reorder=False)
```
returns a new Cartesian communicator.
<br><br>

Methods for this object include `Get_cart_rank()`, `Get_coords()`, *etc.*, which are methods of the class `Cartcomm`.

In [ ]:
# %load cartesian.py
"""Creating a Cartesian Communicator"""

import sys
from mpi4py import MPI

def createCart (parent):
    """Create a 2-d Cartesian communicator from parent communicator"""

    dims = MPI.Compute_dims (parent.size, 2)

    comm = parent.Create_cart (dims, periods = [True, False])
    rank = comm.Get_rank ()
    coords = comm.Get_coords (rank)
    upx = comm.Shift (0, 1)
    upy = comm.Shift (1, 1)

    out = "Rank{:2d} coords{:2d} {:2d} upx(src,dst) {} upy(src,dst) {}\n"\
                         .format(rank, coords[0], coords[1], upx, upy)
    sys.stdout.write (out)

if __name__ == "__main__":
    """Execute in MPI.COMM_WORLD"""
    createCart (MPI.COMM_WORLD)


In [ ]:
!mpirun -np 4 python ./cartesian.py

## MPI communication
<hr style="border: solid 4px green">

### Message passing
* the transaction by which one process accesses data from another process
* all communication has one (or several) sender(s) and one (or several) receiver(s)
<br><br>

### Patterns of communication
* *point-to-point message passing* -- communication between only two processes
* *collective message passing* -- communication between all processes (in a communicator)
  * *one-to-many*
  * *many-to-one*
  * *all-to-all*
<br><br>

### Blocking or nonblocking?
* *blocking messaging*: sender and receiver wait till the receive / send transaction finishes
* *nonblocking messaging*: immediate send, no waiting, need to probe if a message arrived

## <span style="font-family: Courier New, Courier, monospace;">mpi4py</span> communication
<hr style="border: solid 4px green">

### <span style="font-family: Courier New, Courier, monospace;">mpi4py</span> supports
* *fast* (near C-speed) communication of contiguous memory objects (typically, `NumPy` arrays)
* *convenient* communication of generic Python objects (pickling behind the scenes)
<br><br>

### Communication of contiguous memory objects
* function names start with upper-case: `Send()`, `Recv()`, `Bcast()`, `Scatter()`, etc.
* the data communicated is *passed* as an argument to the function
* arguments are explicitly specified, *e.g.* `[data, MPI.DOUBLE]` or `[data, count, MPI.DOUBLE]`
* automatic MPI datatype discovery for NumPy arrays is supported, but limited to basic C types (all C/C99-native signed/unsigned integer types and single/double precision real/complex floating types)
<br><br>


### Communication of generic Python objects
* function names start with lower-case: `send()`, `recv()`, `bcast()`, `scatter()`, etc.
* any object to be communicated is *passed* as a paramenter to the function
* the *received* object is the return value of the function

## Point-to-point communication: blocking <span style="font-family: Courier New, Courier, monospace;">Send()</span> and <span style="font-family: Courier New, Courier, monospace;">Recv()</span>
<hr style="border: solid 4px green">

### <span style="font-family: Courier New, Courier, monospace;">Send()</span> and <span style="font-family: Courier New, Courier, monospace;">Recv()</span>
* communicate buffer-like (contiguous memory) objects
* `Comm.Send (self, buf, int dest, int tag=0)`
  * returns only when the data in the send buffer can be safely changed
  * that does not mean the data arrived at destination
* `Comm.Recv (self, buf, int source=ANY_SOURCE, int tag=ANY_TAG, Status status=None)`
  * returns only when receive buffer contains data expected
<br><br>

### Observations
* the interface makes significant use of optional arguments
* unlike in the C/Fortran standard, count or data type arguments associated with the message buffer are optional as they can be inferred

## Point-to-point communication: blocking <span style="font-family: Courier New, Courier, monospace;">Send()</span> and <span style="font-family: Courier New, Courier, monospace;">Recv()</span> (cont'd)
<hr style="border: solid 4px green">

### The need to communicate contiguous arrays of data occurs frequently
```python
import numpy
sz = 1000
buf = numpy.zeros(sz, type = numpy.double)

if rank == 0:
    MPI.COMM_WORLD.Send ([buf, sz, MPI.DOUBLE], 1, tag = 99)
elif rank == 1:
    MPI.COMM_WORLD.Recv ([buf, sz, MPI.DOUBLE], source = 0, tag = 99)
```

### Count and data type are explicitly specified as part of a list but can be omitted.

## Point-to-point communication: blocking <span style="font-family: Courier New, Courier, monospace;">Send()</span> and <span style="font-family: Courier New, Courier, monospace;">Recv()</span> example
<hr style="border: solid 4px green">

In [ ]:
# %load send_recv_blocking.py
"""A simple blocking Send/Recv pair"""

from mpi4py import MPI
import numpy

def main (comm):
    """Send a message between ranks 0 and 1"""

    # buffer length
    blen = 4
    # process rank
    rank = comm.Get_rank()

    if rank == 0:
        # create send buffer
        buf = numpy.ones (blen, numpy.double)
        # send buffer
        comm.Send ( [buf, blen, MPI.DOUBLE], dest = 1, tag = 999 )
    elif rank == 1:
        # allocate space for recv buffer
        buf = numpy.empty (blen, numpy.double)
        # receive buffer
        comm.Recv ( [buf, blen, MPI.DOUBLE], source = 0, tag = 999 )
        print " rank 1 received: %s" % (buf)

if __name__ == "__main__":
    main (MPI.COMM_WORLD)


In [2]:
! mpirun -np 2 python ./send_recv_blocking.py

 rank 1 received: [ 1.  1.  1.  1.]


## Point-to-point communication: <span style="font-family: Courier New, Courier, monospace;">Send()</span> and <span style="font-family: Courier New, Courier, monospace;">Recv()</span> pairs can lead to deadlocks
<hr style="border: solid 4px green">

### Deadlocks occur when the message passing cannot be completed
* consider the classical example
```python
if rank == 0:
    comm.Send ( [buf, blen, MPI.DOUBLE], dest = 1,   tag = 999 )
    comm.Recv ( [buf, blen, MPI.DOUBLE], source = 1, tag = 999 )
elif rank == 1:
    comm.Send ( [buf, blen, MPI.DOUBLE], dest = 0,   tag = 999 )
    comm.Recv ( [buf, blen, MPI.DOUBLE], source = 0, tag = 999 )
```
* `comm.Send` does not complete until the corresponding `comm.Recv` is posted and *vice versa*
* `comm.Send` will never complete and the program will deadlock

## Point-to-point communication: <span style="font-family: Courier New, Courier, monospace;">Send()</span> and <span style="font-family: Courier New, Courier, monospace;">Recv()</span> pairs can lead to deadlocks (cont'd)
<hr style="border: solid 4px green">

### Solutions
* reverse the order of one of the send/receive pairs
* *equivalent* order is send/receive for even ranks and reversed for odd
* use `Sendrecv`, which performs a blocking send/receive transaction in one go
* use non-blocking send/receive

## Point-to-point communication: non-blocking <span style="font-family: Courier New, Courier, monospace;">Isend()</span> and <span style="font-family: Courier New, Courier, monospace;">Irecv()</span>
<hr style="border: solid 4px green">

### Non-blocking Send and Receive: <span style="font-family: Courier New, Courier, monospace;">Isend()</span> and <span style="font-family: Courier New, Courier, monospace;">Irecv()</span>
* communicate buffer-like (contiguous memory) objects
* `Comm.Isend(self, buf, int dest, int tag=0)`
* `Comm.Irecv(self, buf, int source=ANY_SOURCE, int tag=ANY_TAG)`
* non-blocking (**I** stands for **i**mmediate send/receive)
  * functions return at once, without waiting for transaction to complete
  * functions return an object class `Request`
<br><br>

### <span style="font-family: Courier New, Courier, monospace;">Request</span> objects
* used to ensure the communication took place
* handled by
  * instance methods
```python
request.Wait (self, Status status=None)
```
  * or class methods
```python
Request.Waitall (type cls, requests, statuses=None)
```

## Point-to-point communication: non-blocking <span style="font-family: Courier New, Courier, monospace;">Isend()</span> and <span style="font-family: Courier New, Courier, monospace;">Irecv()</span> (cont'd)
<hr style="border: solid 4px green">

### What is the point?
<br><br>

### Hiding latencies by overlapping communication with processing
* start non-blocking communication using `Isend()` and `Irecv()`
* carry out some processing (which does *not* depend on data communicated)
* handle the requests generated by `Isend()` and `Irecv()` to ensure communication takes place
* carry on with processing that *does* depend on data communicated

## Point-to-point communication: non-blocking <span style="font-family: Courier New, Courier, monospace;">Isend()</span> and <span style="font-family: Courier New, Courier, monospace;">Irecv()</span> example
<hr style="border: solid 4px green">

In [ ]:
# %load isend_irecv_nonblocking.py
from mpi4py import MPI
import numpy

def main (comm):
    """Exchange messages with 'ajoining' ranks"""
    # right-hand adjoining rank (wraparound)
    p1 = comm.rank + 1
    if p1 >= comm.size: p1 = 0
    # left-hand adjoining rank (wraparound)
    m1 = comm.rank - 1
    if m1 < 0: m1 = comm.size - 1

    # send and recv buffers
    smsg = numpy.array ( [comm.rank], numpy.int )
    rmsg = numpy.zeros ( 2, numpy.int )

    # initiate communication
    reqs1 = comm.Isend (smsg, p1)
    reqs2 = comm.Isend (smsg, m1)
    reqr1 = comm.Irecv (rmsg[0:], source = p1)
    reqr2 = comm.Irecv (rmsg[1:], source = m1)

    # receive requests handled by Wait()
    reqr1.Wait()
    reqr2.Wait()

    # all processes print
    # NB: safe to print as messages were received already
    print "[%d] %s %s %s" % (comm.rank, smsg, rmsg[1], rmsg[0])

    # send requests handled by Waitall()
    MPI.Request.Waitall ( [reqs1, reqs2] )


if __name__ == "__main__":

    main(MPI.COMM_WORLD)


In [3]:
! mpirun -np 4 python isend_irecv_nonblocking.py

[2] [2] 1 3
[0] [0] 3 1
[3] [3] 2 0
[1] [1] 0 2


## Point-to-point communication: Python objects
<hr style="border: solid 4px green">

### Message passing of serialised (pickled) Python objects
```python
Comm.send (self, obj, int dest, int tag=0)
Comm.recv (self, buf=None, int source=ANY_SOURCE, int tag=ANY_TAG, Status status=None)
```
<br><br>

### Observations
* serialisation / deserialisation carry an overhead (memory & time)
* complex / large objects are slow to communicate
<br><br>

### Note
* the incoming message is received as the return value
```python
msg = comm.recv(...)
```

## Point-to-point communication: Python objects example
<hr style="border: solid 4px green">

In [ ]:
# %load send_recv_list.py
from mpi4py import MPI
import sys

def main (comm):
    """Send a list from rank 0 to rank 1"""

    if comm.size != 2:
        sys.stdout.write ("Only two processes allowed\n")
        comm.Abort(1)

    if comm.rank == 0:
        msg = ["Any", "old", "thing", comm.rank, {"size" : comm.size}]
        comm.send (msg, dest=1, tag = 999)
    elif comm.rank == 1:
        msg = comm.recv (source=0, tag = 999)
        print " rank 1 received: %s" % (msg)


if __name__ == "__main__":
    main(MPI.COMM_WORLD)


In [4]:
! mpirun -np 2 python send_recv_list.py

 rank 1 received: ['Any', 'old', 'thing', 0, {'size': 2}]


## Collective communication
<hr style="border: solid 4px green; ">

### Multiple processes within same communicator exchange messages
* *always* blocking
* *all* processes in the communicator must call the collective
* failing that, deadlocks occur
<br><br>

### Most useful
* Broadcasts
* Scatter / Gather
* Reductions

## Collective communication (cont'd)
<hr style="border: solid 4px green; ">

### What is the point of collectives?
* specialised functionality than can be replicated using point-to-point functions
* example: a broadcast from rank 0 can be
```python
if comm.rank == 0:
    for i in range (1, comm.size):
        comm.Send ([buf, sz, MPI.DOUBLE], 1, tag = 99)
else:
    comm.Recv ([buf, sz, MPI.DOUBLE], source = 0, tag = 99)
```
<br><br>

### Collectives are implemented as a tree-based communication
* efficient -- $\log N$ instead of $N$
* accurate -- tree reductions are more accurate

## Collective communication: broadcast
<hr style="border: solid 4px green; ">

### One process sends the same message to all other processes in the communicator
<img src="./images/bcast.png"; alt="Logo" style="float: center; width: 40%">
* continuous buffer (*e.g.* `numpy.ndarray`)
```python
comm.Bcast (self, buf, int root=0)
```
* Python objects (pickling behind the scenes)
```python
comm.bcast (self, buf, int root=0)
```

## Collective communication: broadcast example
<hr style="border: solid 4px green; ">

In [ ]:
# %load bcast.py
from mpi4py import MPI
import numpy

def main (comm):
    if comm.rank == 0:
        # rank 0 has ndarray data
        u = numpy.arange (6, dtype=numpy.float64)
        # and dict data
        d = {"x": 1, "y": 3.14, "z": 1-2j}
    else:
        # all other have an empty array
        u = numpy.empty (6, dtype=numpy.float64)
        # and a place-holder
        d = None

    # broadcast from rank 0 to everybody
    comm.Bcast ( [u, MPI.DOUBLE] , root=0 )
    d = comm.bcast ( d, root=0 )

    # all processes print data
    print "[%d] %s %s" % (comm.rank, u, d["y"])


if __name__ == "__main__":
    main (MPI.COMM_WORLD)


In [ ]:
! mpirun -np 4 python bcast.py

## Collective communication: scatter
<hr style="border: solid 4px green; ">

### One process sends a different message to each other processes in the communicator
<img src="./images/scatter.png"; alt="Logo" style="float: center; width: 40%">
* continuous buffer (*e.g.* `numpy.ndarray`)
```python
comm.Scatter (self, sendbuf, recvbuf, int root=0)
```
* Python objects (pickling behind the scenes)
```python
comm.scatter (self, buf, int root=0)
```

## Collective communication: scatter example
<hr style="border: solid 4px green; ">

In [ ]:
# %load scatter.py
from mpi4py import MPI
import numpy

def main (comm):
    if comm.rank == 0:
        # rank 0 has ndarray data
        sendBuf = numpy.empty ([comm.size, 8], dtype=int)
        sendBuf.T[:,:] = range(comm.size)
        # and list data
        obj = [(i+1)**2 for i in range(comm.size)]
    else:
        # all other ranks have an empty send buffer
        sendBuf = None
        # and a place-holder
        obj = None

    # all ranks have a receiving buffer 
    recvBuf = numpy.empty (8, dtype=int)

    # broadcast from rank 0 to everybody
    comm.Scatter (sendBuf, recvBuf, root=0)
    obj = comm.scatter ( obj, root=0 )

    # check: all processes verify data
    print "[%d] %s %s" % (comm.rank, recvBuf, obj)
    # check: cleverer way is to use assert
#   assert numpy.allclose (recvBuf, comm.rank)
#   assert obj == (comm.rank+1)**2

if __name__ == "__main__":
    main (MPI.COMM_WORLD)


In [ ]:
! mpirun -np 4 python scatter.py

## Collective communication: gather
<hr style="border: solid 4px green; ">

### One process receives a different message from each other processes in the communicator
<img src="./images/gather.png"; alt="Logo" style="float: center; width: 40%">
* continuous buffer (*e.g.* `numpy.ndarray`)
```python
comm.Gather (self, sendbuf, recvbuf, int root=0)
```
* Python objects (pickling behind the scenes)
```python
comm.gather (self, buf, int root=0)
```

## Collective communication: gather example
<hr style="border: solid 4px green; ">

In [ ]:
# %load gather.py
from mpi4py import MPI
import numpy

def main (comm):
    # ndarray data to send
    sendBuf = numpy.zeros(8, dtype=int) + comm.rank
    # ndarray data to receive
    if comm.rank == 0:
        recvBuf = numpy.empty ([comm.size, 8], dtype=int)
    else:
        recvBuf = None
    # Python object
    obj = (comm.rank+1)**2

    # gather ndarray
    comm.Gather (sendBuf, recvBuf, root=0)
    # gather Python objects
    obj = comm.gather(obj, root=0)

    # check: all ranks print verify data
    print "[%d] %s %s" % (comm.rank, recvBuf, obj)


if __name__ == "__main__":
    main (MPI.COMM_WORLD)


In [ ]:
! mpirun -np 4 python gather.py

## Collective communication: reduction
<hr style="border: solid 4px green; ">

### Data in all processes is "reduced" to a single process according to an operation
* operations are defined by the `Op` class in `mpi4py`
  * pre-defined operations: `MIN`, `MAX`, `SUM`, etc.
  * user defined operations can be programmed for
<br><br>

### Example
* find the maximum value on all the data across all processes
  * each process finds a "local" maximum across its own data
  * all local maxima are "reduced" to a single value (the maximum across all data)
<br><br>

### Reductions on <span style="font-family: Courier New, Courier, monospace;">NumPy</span> arrays
* (memory-contiguous) buffers
```python
Comm.Reduce (self, sendbuf, recvbuf, Op op=SUM, int root=0)
```
* Python objects
```python
Comm.reduce (self, sendobj, Op=SUM, int root=0)
```

## Collective communication: reduction example
<hr style="border: solid 4px green; ">

In [ ]:
# %load reduce.py
from mpi4py import MPI
import numpy

def main(comm):

    # ndarray data
    buf  = ( numpy.ones (6, dtype=int) + 1 ) * comm.rank
    buf2 = numpy.empty (6, dtype=int)

    # Python object data
    obj  = [ comm.rank ]

    # reduction on ndarray: buf reduced to buff2
    # comm.Reduce ( buf, buf2, op=MPI.SUM, root=0 ) # <- this works too
    comm.Reduce ( [buf, MPI.INT], [buf2, MPI.INT], op=MPI.SUM, root=0 )

    # reduction of Python object
    obj  = comm.reduce ( [comm.rank], op=MPI.SUM, root=0 )

    # all processes print data
    if comm.rank == 0:
        print "[%d] %s %s <- reduced" % (comm.rank, buf2, obj)
    else:
        print "[%d] %s %s" % (comm.rank, buf, obj)

if __name__ == "__main__":
    main(MPI.COMM_WORLD)


In [5]:
! mpirun -np 4 python reduce.py

[3] [6 6 6 6 6 6] None
[1] [2 2 2 2 2 2] None
[2] [4 4 4 4 4 4] None
[0] [12 12 12 12 12 12] [0, 1, 2, 3] <- reduced


## Other features
<hr style="border: solid 4px green">

### Standard MPI functionality is available (<span style="font-family: Courier New, Courier, monospace;">mpi4py</span> version 2.0)
* `comm.Abort()` (terminate MPI execution environment) , `MPI.Wtime()` (accurate elapsed time), ...
* user-defined types (methods implemented in the `Datatype` class)
* RMA (remote single-sided) communication (see the `Win` class)
* MPI-IO (see the `File` class)
* a few functions are not implemented, *e.g.* `AlltoAllw()` (all processes send data of different types to, and receive data of different types from, all processes)

## Some other considerations
<hr style="border: solid 4px green">

### Performance notes
* problem: `import` statements trigger a lot of small-file I/O
  * in parallel calculations, all proceses perform same I/O
  * large numbers of MPI tasks can cause severe logjam at `import`
  * a serious obstacle to Python scaling
* solution #1: install modules read by `import` in ramdisk
* solution #2: start in Python and call another language
  * can typically pass a communicator and other relevant data
  * convenient access to C `MPI_Comm` handles requires `mpi4py` 2.0

## Summary
<hr style="border: solid 4px green; ">

### We have briefly looked at
* the message passing paradigm and the MPI standard
* the `mpi4py` module in Python
<br><br>

###  <span style="font-family: Courier New, Courier, monospace;">mpi4py</span> for production code?
* yes, if
  * communication is not very frequent
  * communication does not involve a lot of data
  * performance is not the primary concern
* no, if
  * the algorithm requires a lot of communication
  * the plan is to scale to large core counts

### A good idea (?)
* `mpi4py` may be a good tool for teaching the basic concepts of distributed (MPI) computing

<img src="../../images/reusematerial.png"; style="float: center; width: 90"; >